In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
import os
from dotenv import load_dotenv
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import InMemorySaver
load_dotenv()

True

In [95]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")

In [ ]:
model = ChatOpenAI(
    model="gpt-5",
    temperature=0.1,
    timeout=30
)



In [97]:
SYSTEM_PROMPT = """You are an expert vegan/plant-based nutritionist and meal generator.
Your name is House of PlantAgent.

Rules:
- Think step-by-step.
- Provide a list of ingredients, quantities, and cooking instructions in return to the user query for recipe, based on your search result.
- If the user asks non-recipe related questions, state that your purpose is only to generate user recipes.

You have access to one tool, which you MUST use before generating each new recipe-generated response:

- tavily_search_tool: use this to search for a vegan recipe based on the user's stated preferences.
"""

In [ ]:
tavily_search_tool = TavilySearch(
        max_results=1,
        topic="general",
        search_depth = "advanced",
    )



In [102]:
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

In [103]:
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # The response containing the title, ingredients, instructions.
    vegan_recipe: str
    # The URL from `tavily_search_tool`
    recipe_url: str

In [99]:
agent = create_agent(
    model=model,
    system_prompt = SYSTEM_PROMPT,
    tools=[tavily_search_tool],
    

)

In [100]:
user_input = "What's a quick tofu and noodles recipe?"

In [101]:
for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's a quick tofu and noodles recipe?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_mcfHXud4ZQ8emPean3sqhCKz)
 Call ID: call_mcfHXud4ZQ8emPean3sqhCKz
  Args:
    query: quick vegan tofu and noodles recipe 15-minute stir-fry
================================= Tool Message =================================
Name: tavily_search

{"query": "quick vegan tofu and noodles recipe 15-minute stir-fry", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://munchmealsbyjanet.com/2020/05/06/vegan-stir-fry-noodles-with-tofu/", "title": "Vegan Stir Fry Noodles with Tofu - Munchmeals by Janet", "content": "During last few minutes, add a splash of soy sauce to the pan and sprinkle tofu with garlic powder. Remove from pan and set aside.\n While tofu is cooking, whisk together sauce ingredients in a bowl.\n Add the other 1

KeyboardInterrupt: 

Works but it doesn't look like the final generation meaningfully uses the tool's output...